### Important 
This notebook is used to run code (like a main.py) __only__.

All methods must be written in separate .py files. 

Allowed here: Imports, data loading, method calling. 

In order for the first part to work, we assume that the run.sh has been executed.

# Building a Text Classifier

Imports

In [1]:
import pandas as pd
import sklearn as sk
import numpy as np
import matplotlib
import pickle as pkl

Constants

In [2]:
DATA_FOLDER = 'data/'
TWITTER_FOLDER = 'twitter-datasets/'

### 1. Construct Features for the Training Texts
Load the training tweets and the built GloVe word embeddings. 

In [3]:
#Load partial datasets for now. 
#read_table automatically uses the new line as a separator. 
train_neg = open(DATA_FOLDER+TWITTER_FOLDER+"train_neg.txt").readlines()
train_pos = open(DATA_FOLDER+TWITTER_FOLDER+"train_pos.txt").readlines()

In [4]:
#Load embeddings
embeddings = np.load('embeddings.npy')

In [5]:
#Load vocabulary
with open('vocab.pkl', 'rb') as f:
        vocab = pkl.load(f)

Using the word embeddings, construct a feature representation of each training tweet (by averaging the word vectors over all words of the tweet).

In [ ]:
def feature_representation(data):
    tot_words = 0
    nb_lines =  np.shape(data)[0]
    word_count = np.zeros(nb_lines)
    for i in range(nb_lines):
        #array of words for tweet i
        words = data[i].split()
        #number of words in word array
        nb_words =  len(words)
        #set word count for word vector i
        word_count[i] = nb_words
        #increment total number of tweets
        tot_words += nb_words
    #return average of word vectors over all words of the tweet
    return word_count/tot_words

In [30]:
features_neg = feature_representation(train_neg)
features_pos = feature_representation(train_pos)